In [ ]:
# Importando as Bibliotecas Básicas
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')
# Importando a Biblioteca do KNN
from sklearn.neighbors import KNeighborsClassifier
# Importando a Biblioteca de Acurácia, Matriz de Confusão (em quantos jogos entrou em quantos foi green ou red)
from sklearn.metrics import accuracy_score, confusion_matrix
# Importando a Biblioteca SkLearn para padronizar os dados (entre 0 e 1)
from sklearn.preprocessing import StandardScaler
# Importando a Biblioteca SkLearn para dividir os dados entre treinamento e teste
from sklearn.model_selection import train_test_split

# Importando a Classe Cores
class bcolors:
    OKGREEN = '\033[92m' #GREEN
    OkBLUE = '\033[94m' #BLUE
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [ ]:
# Importando os DataFrames
df_BR13 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2013.xlsx?raw=true')
df_BR14 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2014.xlsx?raw=true')
df_BR15 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2015.xlsx?raw=true')
df_BR16 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2016.xlsx?raw=true')
df_BR17 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2017.xlsx?raw=true')
df_BR18 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2018.xlsx?raw=true')
df_BR19 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2019.xlsx?raw=true')
df_BR20 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2020.xlsx?raw=true')
df_BR21 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2021.xlsx?raw=true')
df_BR22 = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/Brazil%20Serie%20A_2022.xlsx?raw=true')

# Concatenando os DataFrames
df = [df_BR13,df_BR14,df_BR15,df_BR16,df_BR17,df_BR18,df_BR19,df_BR20,df_BR21]
df = pd.concat(df)
df = df[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]
df = df[(df.FT_Odds_H != 0) & (df.FT_Odds_D != 0) & (df.FT_Odds_A != 0)]
df = df.dropna()
# Ajustando o Índice
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

In [ ]:
df

In [ ]:
# Dados de Pontos
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'Result_FT'] = "H"
df.loc[(df['FT_Goals_H'] == df['FT_Goals_A']), 'Result_FT'] = "D"
df.loc[(df['FT_Goals_H'] < df['FT_Goals_A']), 'Result_FT'] = "A"

df.loc[(df['Result_FT'] == "H"), 'Pontos_H'] = 3
df.loc[(df['Result_FT'] == "D"), 'Pontos_H'] = 1
df.loc[(df['Result_FT'] == "A"), 'Pontos_H'] = 0

df.loc[(df['Result_FT'] == "A"), 'Pontos_A'] = 3
df.loc[(df['Result_FT'] == "D"), 'Pontos_A'] = 1
df.loc[(df['Result_FT'] == "H"), 'Pontos_A'] = 0

# Estatísticas de Pontos
# Média
df['Media_Pontos_H'] = df.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Pontos_A'] = df.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
# Desvio Padrão
df['DP_Pontos_H'] = df.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df['DP_Pontos_A'] = df.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
# Coeficiente de Variação
df['CV_Pontos_H'] = (df['DP_Pontos_H'] / df['Media_Pontos_H']).shift(1)
df['CV_Pontos_A'] = (df['DP_Pontos_A'] / df['Media_Pontos_A']).shift(1)

df.replace(np.inf, 1, inplace=True)
df.dropna(inplace=True)

In [ ]:
# Criando a Coluna de 0 e 1 para análise do Modelo de Machine Learning
df.loc[(df['FT_Goals_H'] > df['FT_Goals_A']), 'BackHome'] = 1
df.loc[(df['FT_Goals_H'] <= df['FT_Goals_A']), 'BackHome'] = 0

In [ ]:
df

In [ ]:
df1 = df[['FT_Odds_H','FT_Odds_D','FT_Odds_A','Media_Pontos_H','CV_Pontos_H','Media_Pontos_A','CV_Pontos_A','BackHome']]

In [ ]:
df1 = df1.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df1_x = df1.iloc[:, :-1].values
df1_y = df1.iloc[:, -1].values

In [ ]:
df1_x

In [ ]:
df1_y

In [ ]:
# Definições de Parâmetros e dados de entrada e resultado para treinamento
knn_dados = KNeighborsClassifier(n_neighbors=3, metric="minkowski", p=2)
knn_dados.fit(df1_x, df1_y)

In [ ]:
df2 = df_BR22[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Goals_H','FT_Goals_A']]
df2 = df2[(df2.FT_Odds_H != 0) & (df2.FT_Odds_D != 0) & (df2.FT_Odds_A != 0)]
df2 = df2.dropna()
# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

# Dados de Pontos
df2.loc[(df2['FT_Goals_H'] > df2['FT_Goals_A']), 'Result_FT'] = "H"
df2.loc[(df2['FT_Goals_H'] == df2['FT_Goals_A']), 'Result_FT'] = "D"
df2.loc[(df2['FT_Goals_H'] < df2['FT_Goals_A']), 'Result_FT'] = "A"

df2.loc[(df2['Result_FT'] == "H"), 'Pontos_H'] = 3
df2.loc[(df2['Result_FT'] == "D"), 'Pontos_H'] = 1
df2.loc[(df2['Result_FT'] == "A"), 'Pontos_H'] = 0

df2.loc[(df2['Result_FT'] == "A"), 'Pontos_A'] = 3
df2.loc[(df2['Result_FT'] == "D"), 'Pontos_A'] = 1
df2.loc[(df2['Result_FT'] == "H"), 'Pontos_A'] = 0

# Estatísticas de Pontos
# Média
df2['Media_Pontos_H'] = df2.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df2['Media_Pontos_A'] = df2.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
# Desvio Padrão
df2['DP_Pontos_H'] = df2.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df2['DP_Pontos_A'] = df2.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
# Coeficiente de Variação
df2['CV_Pontos_H'] = (df2['DP_Pontos_H'] / df2['Media_Pontos_H']).shift(1)
df2['CV_Pontos_A'] = (df2['DP_Pontos_A'] / df2['Media_Pontos_A']).shift(1)

df2.replace(np.inf, 1, inplace=True)
df2.dropna(inplace=True)

#Salvando o DF
dfdf = df2

# Criando a Coluna de 0 e 1 para análise do Modelo de Machine Learning
df2.loc[(df2['FT_Goals_H'] > df2['FT_Goals_A']), 'BackHome'] = 1
df2.loc[(df2['FT_Goals_H'] <= df2['FT_Goals_A']), 'BackHome'] = 0

df2 = df2[['FT_Odds_H','FT_Odds_D','FT_Odds_A','Media_Pontos_H','CV_Pontos_H','Media_Pontos_A','CV_Pontos_A','BackHome']]

In [ ]:
df2

In [ ]:
df2 = df2.dropna()
# convertendo os arrays (matriz)
# definindo qual coluna é a entrada e qual é a saída
df2_x = df2.iloc[:, :-1].values

# Previsões
previsoes_knn2 = knn_dados.predict(df2_x)

df2["Previsões"] = previsoes_knn2

stake = 1
win_Back = stake * (df2.FT_Odds_H - 1)
lose_Back = -stake

df2.loc[(df2['Previsões'] == 1) & (df2['BackHome'] == 1), 'Profit'] = win_Back
df2.loc[(df2['Previsões'] == 1) & (df2['BackHome'] == 0), 'Profit'] = lose_Back
df2.loc[(df2['Previsões'] == 0) & (df2['BackHome'] == 1), 'Profit'] = 0
df2.loc[(df2['Previsões'] == 0) & (df2['BackHome'] == 0), 'Profit'] = 0

filtro = df2.Previsões == 1
df0 = df2[filtro]

# Ajustando o Índice
df0.reset_index(inplace=True, drop=True)
df0.index = df0.index.set_names(['Nº'])
df0 = df0.rename(index=lambda x: x + 1)

df0['Profit_acu'] = df0.Profit.cumsum()
df0.Profit_acu.plot()
df0.Profit_acu.tail(1)

In [ ]:
dfdf = dfdf[['Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','Media_Pontos_H','CV_Pontos_H','Media_Pontos_A','CV_Pontos_A']]
dfdf = dfdf.tail(10)
dfdf

In [ ]:
for a,b,c,d,e,f,g,h,i in zip(dfdf.Home,dfdf.Away,dfdf.FT_Odds_H,dfdf.FT_Odds_D,dfdf.FT_Odds_A,dfdf.Media_Pontos_H,dfdf.CV_Pontos_H,dfdf.Media_Pontos_A,dfdf.CV_Pontos_A):
  
    Home = a
    Away = b
    FT_Odds_H = c
    FT_Odds_D = d
    FT_Odds_A = e
    Media_Pontos_H = f
    CV_Pontos_H = g
    Media_Pontos_A = h
    CV_Pontos_A = i
    
    Odds = [FT_Odds_H,FT_Odds_D,FT_Odds_A,Media_Pontos_H,CV_Pontos_H,Media_Pontos_A,CV_Pontos_A]

    # Transformando a Lista em uma Matriz
    odds_jogo = np.array(Odds)

    previsao_jogo_knn = odds_jogo.reshape(1, -1)
    previsao_jogo_knn = knn_dados.predict(previsao_jogo_knn)

    print('');print(Home,'x',Away)

    if previsao_jogo_knn == 1:
      print(bcolors.OKGREEN + "Fazer Back Mandante" + bcolors.RESET)
    else:
      print(bcolors.FAIL + "Ficar de Fora desse jogo" + bcolors.RESET)

